# faimed3d wrapper for Learner class

In [ ]:
# hide
import sys
sys.path.append("..")

In [ ]:
# export
# default_exp learner
from fastai.basics import *
from fastai.callback.all import *

In [ ]:
# export
from faimed3d.basics import *
from faimed3d.augment import *
from faimed3d.preprocess import *
from faimed3d.layers import *
from faimed3d.data import *
from faimed3d.models.unet import DynamicUnet3D
from faimed3d.models.losses import DiceLoss
from fastai.vision.learner import _default_meta, _add_norm, model_meta, create_body, create_unet_model

In [ ]:
d =  pd.read_csv('../../mri-spa/data/train_knee.csv')
dls = ImageDataLoaders3D.from_df(d, path='/media/..', 
                                 item_tfms = ResizeCrop3D(crop_by = (0., 0.1, 0.1), resize_to = (20, 150, 150), perc_crop = True),
                                 bs = 2, 
                                 val_bs = 2)

No rescale method was used. This is not advisable due to high risk of exploding gradients. Falling back to mean scaling.


## CNN learner
The fastai cnn_learner class can be used to construct a 3D CNN with only a few modifactions to the original functions. See the `faimed3d.layers` notebook for details. 

In [ ]:
# export
@delegates(Learner.__init__)
def cnn_learner_3d(dls, arch, loss_func=None, pretrained=True, cut=None, splitter=None,
                y_range=None, config=None, n_out=None, normalize=True, **kwargs):
    """
    Build a convnet style learner from `dls` and `arch`
    Same as fastai func but adds the `AddColorChannel` callback.
    """
    if config is None: config = {}
    meta = model_meta.get(arch, _default_meta)
    if n_out is None: n_out = get_c(dls)
    assert n_out, "`n_out` is not defined, and could not be inferred from data, set `dls.c` or pass `n_out`"
    # if normalize: _add_norm(dls, meta, pretrained) # no effect as no TenosrImage is passed in 3d
    if y_range is None and 'y_range' in config: y_range = config.pop('y_range')
    model = create_cnn_model(arch, n_out, ifnone(cut, meta['cut']), pretrained, y_range=y_range, **config)
    learn = Learner(dls, model, loss_func=loss_func, splitter=ifnone(splitter, meta['split']), **kwargs)
    if pretrained: learn.freeze()
    return learn

In [ ]:
from torchvision.models.video import r3d_18
learn = cnn_learner_3d(dls, r3d_18, model_dir='/home/bressekk/Documents/faimed3d/nbs')

In [ ]:
#slow
learn.fine_tune(1, 0.001)

epoch,train_loss,valid_loss,time


## UNet Learner

In [ ]:
# export
@delegates(DynamicUnet3D.__init__)
def create_unet_model_3d(arch, n_out, img_size, pretrained=True, cut=None, n_in=3, **kwargs):
    "Create custom unet architecture"
    meta = model_meta.get(arch, _default_meta)
    body = create_body(arch, n_in, pretrained, ifnone(cut, meta['cut']))
    model = DynamicUnet3D(body, n_out, img_size, **kwargs)
    return model

In [ ]:
# export
@delegates(create_unet_model)
def unet_learner_3d(dls, arch, normalize=True, n_out=None, pretrained=True, config=None,
                 # learner args
                 loss_func=None, opt_func=Adam, lr=defaults.lr, splitter=None, cbs=None, metrics=None, path=None,
                 model_dir='models', wd=None, wd_bn_bias=False, train_bn=True, moms=(0.95,0.85,0.95),
                 # other model args
                 **kwargs):
    "Build a unet learner from `dls` and `arch`"

    if config:
        warnings.warn('config param is deprecated. Pass your args directly to unet_learner.')
        kwargs = {**config, **kwargs}

    meta = model_meta.get(arch, _default_meta)
    # if normalize: _add_norm(dls, meta, pretrained) # no effect as no TenosrImage is passed in 3d

    n_out = ifnone(n_out, get_c(dls))
    assert n_out, "`n_out` is not defined, and could not be inferred from data, set `dls.c` or pass `n_out`"
    img_size = dls.one_batch()[0].shape[-3:]
    assert img_size, "image size could not be inferred from data"
    model = create_unet_model_3d(arch, n_out, img_size, pretrained=pretrained, **kwargs)
    
    if loss_func is None: loss_func = DiceLoss(smooth=0.)
    splitter=ifnone(splitter, meta['split'])
    learn = Learner(dls=dls, model=model, loss_func=loss_func, opt_func=opt_func, lr=lr, splitter=splitter, cbs=cbs,
                   metrics=metrics, path=path, model_dir=model_dir, wd=wd, wd_bn_bias=wd_bn_bias, train_bn=train_bn,
                   moms=moms)
    if pretrained: learn.freeze()
    # keep track of args for loggers
    store_attr('arch,normalize,n_out,pretrained', self=learn)
    if kwargs: store_attr(self=learn, **kwargs)
    return learn

In [ ]:
prostate = pd.read_csv('../../dl-prostate-mapping/data/prostata-train.csv', sep = ',')[['t2_cropped_dcm', 't2_mask_cropped']]
dls = SegmentationDataLoaders3D.from_df(prostate, path='/media/..', 
                                codes = ['void', 'transitional', 'peripheral', 'cancer'],
                                 item_tfms = ResizeCrop3D(crop_by = (0., 0.1, 0.1), resize_to = (20, 100, 100), perc_crop = True), 
                                 bs = 2, val_bs = 2)

In [ ]:
learn = unet_learner_3d(dls, r3d_18, model_dir='/home/bressekk/Documents/faimed3d/nbs')

In [ ]:
#slow
learn.fit_one_cycle(1, 0.001)

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted 01_basics.ipynb.
Converted 02_preprocessing.ipynb.
Converted 03_transforms.ipynb.
Converted 04_dataloaders.ipynb.
Converted 05_layers.ipynb.
Converted 06_learner.ipynb.
Converted 06a_models.alexnet.ipynb.
Converted 06b_models.resnet.ipynb.
Converted 06c_models.densenet.ipynb.
Converted 06d_models.unet.ipynb.
Converted 06e_models.deeplabv3.ipynb.
Converted 06f_models.losses.ipynb.
Converted 07_callback.ipynb.
Converted 98_widgets.ipynb.
Converted 99_tools.ipynb.
Converted 99_widgets.cam.ipynb.
Converted index.ipynb.
Converted tutorial_mri-normalization.ipynb.


###### 